In [2]:
import json

with open("/home/bota/planned_missions/campo_gao.plan", "r") as file:
    plan_data = json.load(file)

print(plan_data)

{'fileType': 'Plan', 'geoFence': {'circles': [], 'polygons': [], 'version': 2}, 'groundStation': 'QGroundControl', 'mission': {'cruiseSpeed': 15, 'firmwareType': 12, 'globalPlanAltitudeMode': 1, 'hoverSpeed': 5, 'items': [{'autoContinue': True, 'command': 530, 'doJumpId': 1, 'frame': 2, 'params': [0, 2, None, None, None, None, None], 'type': 'SimpleItem'}, {'AMSLAltAboveTerrain': 5, 'Altitude': 5, 'AltitudeMode': 1, 'autoContinue': True, 'command': 22, 'doJumpId': 2, 'frame': 3, 'params': [0, 0, 0, None, 41.737333879364094, 12.51367080978099, 5], 'type': 'SimpleItem'}, {'TransectStyleComplexItem': {'CameraCalc': {'AdjustedFootprintFrontal': 2.441463414634146, 'AdjustedFootprintSide': 3.2499999999999996, 'CameraName': 'Custom Camera', 'DistanceMode': 1, 'DistanceToSurface': 5, 'FixedOrientation': False, 'FocalLength': 3, 'FrontalOverlap': 70, 'ImageDensity': 0.33028455284552843, 'ImageHeight': 2464, 'ImageWidth': 3280, 'Landscape': True, 'MinTriggerInterval': 0, 'SensorHeight': 6.5, 'Se

In [3]:
home_position = plan_data['mission']['plannedHomePosition']

In [4]:
n_items = len(plan_data["mission"]["items"])
waypoints = []
image_capture_waypoints = []
survey = []
for i in range(n_items):
    item = plan_data["mission"]["items"][i]
    if "command" in item:
        if item["command"] == 530:
            print("arming")
        if item["command"] == 22:
            print("takeoff to {} m ".format(item["params"][-1]))
    elif "TransectStyleComplexItem" in item:  # survey or corridor scan
        if item["complexItemType"] == "survey":
            print("survey")
            survey = item["TransectStyleComplexItem"]
            for n, waypoint_item in enumerate(survey["Items"]):
                print(waypoint_item)
                if waypoint_item["command"] == 16:  # MAV_CMD_NAV_WAYPOINT
                    waypoints.append(waypoint_item["params"][4:7])
                elif (
                    waypoint_item["command"] == 2000 or waypoint_item["command"] == 206
                ):  # MAV_CMD_IMAGE_START_CAPTURE
                    image_capture_waypoints.append(waypoints[-1])
                else:
                    print(waypoint_item)
                print(
                    "{} => {} way and {} imgs".format(
                        n, len(waypoints), len(image_capture_waypoints)
                    )
                )
            print("camerashots ", survey["CameraShots"])
            # assert survey['CameraShots']==len(image_capture_waypoints)
    else:
        print(item)
    # command 2000 is MAV_CMD_IMAGE_START_CAPTURE
    # 2001 MAV_CMD_IMAGE_STOP_CAPTURE
    # command 16 is MAV_CMD_NAV_WAYPOINT

    # https://mavlink.io/en/messages/common.html

arming
takeoff to 5 m 
survey
{'autoContinue': True, 'command': 16, 'doJumpId': 3, 'frame': 3, 'params': [0, 0, 0, None, 41.73707341957598, 12.513699020104042, 5], 'type': 'SimpleItem'}
0 => 1 way and 0 imgs
{'autoContinue': True, 'command': 16, 'doJumpId': 4, 'frame': 3, 'params': [4, 0, 0, None, 41.73706952540144, 12.513702033069473, 5], 'type': 'SimpleItem'}
1 => 2 way and 0 imgs
{'autoContinue': True, 'command': 2000, 'doJumpId': 5, 'frame': 2, 'params': [0, 0, 1, None, None, None, None], 'type': 'SimpleItem'}
2 => 2 way and 1 imgs
{'autoContinue': True, 'command': 16, 'doJumpId': 6, 'frame': 3, 'params': [4, 0, 0, None, 41.73705051043095, 12.513716745153962, 5], 'type': 'SimpleItem'}
3 => 3 way and 1 imgs
{'autoContinue': True, 'command': 2000, 'doJumpId': 7, 'frame': 2, 'params': [0, 0, 1, None, None, None, None], 'type': 'SimpleItem'}
4 => 3 way and 2 imgs
{'autoContinue': True, 'command': 16, 'doJumpId': 8, 'frame': 3, 'params': [4, 0, 0, None, 41.737037559378926, 12.5137267655

In [5]:
len(waypoints)
print(waypoints[0])

[41.73707341957598, 12.513699020104042, 5]


In [49]:
survey

{'CameraCalc': {'AdjustedFootprintFrontal': 2.441463414634146,
  'AdjustedFootprintSide': 3.2499999999999996,
  'CameraName': 'Custom Camera',
  'DistanceMode': 3,
  'DistanceToSurface': 5,
  'FixedOrientation': False,
  'FocalLength': 3,
  'FrontalOverlap': 70,
  'ImageDensity': 0.33028455284552843,
  'ImageHeight': 2464,
  'ImageWidth': 3280,
  'Landscape': True,
  'MinTriggerInterval': 0,
  'SensorHeight': 6.5,
  'SensorWidth': 6.5,
  'SideOverlap': 70,
  'ValueSetIsDistance': True,
  'version': 2},
 'CameraShots': 43,
 'CameraTriggerInTurnAround': False,
 'HoverAndCapture': False,
 'Items': [{'autoContinue': True,
   'command': 16,
   'doJumpId': 3,
   'frame': 0,
   'params': [0, 0, 0, None, 41.73707341957598, 12.513699020104042, 92],
   'type': 'SimpleItem'},
  {'autoContinue': True,
   'command': 16,
   'doJumpId': 4,
   'frame': 0,
   'params': [0, 0, 0, None, 41.73706952540144, 12.513702033069473, 92],
   'type': 'SimpleItem'},
  {'autoContinue': True,
   'command': 206,
   'd

In [51]:
len(survey["VisualTransectPoints"])

24

In [57]:
import json
with open('/home/bota/planned_missions/campo_gao.plan', 'r') as file:
    mission_plan = json.load(file)

# Extract survey mission details
survey_mission = None
for item in mission_plan['mission']['items']:
    if 'TransectStyleComplexItem' in item:
        survey_mission = item['TransectStyleComplexItem']
        break

# Display extracted survey mission details
if survey_mission:
    camera_settings = survey_mission['CameraCalc']
    camera_shots = survey_mission['CameraShots']
    items = survey_mission['Items']
    
    # Extract waypoints and camera triggers
    waypoints = [item for item in items if item['command'] == 16]
    camera_triggers = [item for item in items if item['command'] == 206]
    
    print("Camera Settings:", json.dumps(camera_settings, indent=4))
    print("Number of Camera Shots:", camera_shots)
    print("Waypoints:")
    for wp in waypoints:
        print(wp)
        # print(json.dumps(wp, indent=4))
    print(len(waypoints))
    print("Camera Triggers:")
    
    for ct in camera_triggers:
        print(ct)
        # print(json.dumps(ct, indent=4))
    print(len(camera_triggers))
else:
    print("No survey mission found in the plan.")


Camera Settings: {
    "AdjustedFootprintFrontal": 2.441463414634146,
    "AdjustedFootprintSide": 3.2499999999999996,
    "CameraName": "Custom Camera",
    "DistanceMode": 3,
    "DistanceToSurface": 5,
    "FixedOrientation": false,
    "FocalLength": 3,
    "FrontalOverlap": 70,
    "ImageDensity": 0.33028455284552843,
    "ImageHeight": 2464,
    "ImageWidth": 3280,
    "Landscape": true,
    "MinTriggerInterval": 0,
    "SensorHeight": 6.5,
    "SensorWidth": 6.5,
    "SideOverlap": 70,
    "ValueSetIsDistance": true,
    "version": 2
}
Number of Camera Shots: 43
Waypoints:
{'autoContinue': True, 'command': 16, 'doJumpId': 3, 'frame': 0, 'params': [0, 0, 0, None, 41.73707341957598, 12.513699020104042, 92], 'type': 'SimpleItem'}
{'autoContinue': True, 'command': 16, 'doJumpId': 4, 'frame': 0, 'params': [0, 0, 0, None, 41.73706952540144, 12.513702033069473, 92], 'type': 'SimpleItem'}
{'autoContinue': True, 'command': 16, 'doJumpId': 6, 'frame': 0, 'params': [0, 0, 0, None, 41.73703

In [60]:
from math import radians, sin, cos, sqrt, atan2

# Load the mission plan from a file


# Haversine formula to calculate distance between two points on the Earth
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # radius of Earth in meters
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon1 - lon2)
    a = sin(dphi / 2) ** 2 + cos(phi1) * cos(phi2) * sin(dlambda / 2) ** 2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))


# Calculate trigger distance based on camera settings
def calculate_trigger_distance(
    altitude, sensor_width, sensor_height, focal_length, overlap_percentage
):
    ground_resolution = (sensor_width * altitude) / (focal_length * 1000)
    trigger_distance = ground_resolution * (1 - overlap_percentage / 100)
    return trigger_distance


# Extract survey mission details
survey_mission = None
for item in mission_plan["mission"]["items"]:
    if "TransectStyleComplexItem" in item:
        survey_mission = item["TransectStyleComplexItem"]
        break

if survey_mission:
    camera_settings = survey_mission["CameraCalc"]
    camera_shots = survey_mission["CameraShots"]
    items = survey_mission["Items"]

    # Extract waypoints and camera triggers
    waypoints = [item for item in items if item["command"] == 16]
    camera_triggers = [item for item in items if item["command"] == 206]

    print(f"Extracted Camera Shots: {camera_shots}")
    print(f"Number of Waypoints: {len(waypoints)}")
    print(f"Number of Camera Triggers: {len(camera_triggers)}")

    # Extract camera parameters
    altitude = survey_mission["CameraCalc"]["DistanceToSurface"]
    sensor_width = survey_mission["CameraCalc"]["SensorWidth"]
    sensor_height = survey_mission["CameraCalc"]["SensorHeight"]
    focal_length = survey_mission["CameraCalc"]["FocalLength"]
    overlap_percentage = survey_mission["CameraCalc"]["SideOverlap"]

    # Calculate trigger distance
    trigger_distance = calculate_trigger_distance(
        altitude, sensor_width, sensor_height, focal_length, overlap_percentage
    )
    print(f"Calculated Camera Trigger Distance: {trigger_distance:.2f} meters")

    # Set the camera trigger command parameters
    for trigger in camera_triggers:
        trigger["params"][1] = trigger_distance  # Set the trigger distance

    # Verify camera triggers and distances
    trigger_distances = [trigger["params"][1] for trigger in camera_triggers]
    print(f"Camera Trigger Distances: {trigger_distances}")

    # Validate the number of camera triggers matches the expected number of shots
    if len(camera_triggers) != camera_shots:
        print(
            f"Warning: Number of camera triggers ({len(camera_triggers)}) does not match the expected camera shots ({camera_shots})."
        )

    # Check if each camera trigger position matches a waypoint position
    waypoint_positions = [(wp["params"][4], wp["params"][5]) for wp in waypoints]
    camera_trigger_positions = [
        (ct["params"][4], ct["params"][5]) for ct in camera_triggers
    ]
    unmatched_triggers = [
        pos for pos in camera_trigger_positions if pos not in waypoint_positions
    ]
    if unmatched_triggers:
        print(
            f"Warning: Unmatched camera trigger positions found: {unmatched_triggers}"
        )
    else:
        print("All camera trigger positions match waypoints.")

    # Save the updated mission plan to a new file
    with open("updated_mission_plan.json", "w") as file:
        json.dump(mission_plan, file, indent=4)
else:
    print("No survey mission found in the plan.")

Extracted Camera Shots: 43
Number of Waypoints: 24
Number of Camera Triggers: 12
Calculated Camera Trigger Distance: 0.00 meters
Camera Trigger Distances: [0.0032500000000000007, 0.0032500000000000007, 0.0032500000000000007, 0.0032500000000000007, 0.0032500000000000007, 0.0032500000000000007, 0.0032500000000000007, 0.0032500000000000007, 0.0032500000000000007, 0.0032500000000000007, 0.0032500000000000007, 0.0032500000000000007]


In [59]:
 survey_mission['CameraCalc']

{'AdjustedFootprintFrontal': 2.441463414634146,
 'AdjustedFootprintSide': 3.2499999999999996,
 'CameraName': 'Custom Camera',
 'DistanceMode': 3,
 'DistanceToSurface': 5,
 'FixedOrientation': False,
 'FocalLength': 3,
 'FrontalOverlap': 70,
 'ImageDensity': 0.33028455284552843,
 'ImageHeight': 2464,
 'ImageWidth': 3280,
 'Landscape': True,
 'MinTriggerInterval': 0,
 'SensorHeight': 6.5,
 'SensorWidth': 6.5,
 'SideOverlap': 70,
 'ValueSetIsDistance': True,
 'version': 2}

In [52]:
len(survey["Items"])

36

In [48]:
# List of MAVLINK commands


# Extract points where pictures are taken (commands with command code 206)
picture_points = []
for command in survey:
    print(command)
    if command['command'] == 206:
        # Extract latitude, longitude, and altitude from params
        lat = command['params'][4]
        lon = command['params'][5]
        alt = command['params'][6]
        picture_points.append((lat, lon, alt))

# Ensure that there are 43 camera shots
if len(picture_points) != 43:
    print("Warning: Number of camera shots is not 43!")

# Print the extracted points
for i, point in enumerate(picture_points, 1):
    print(f"Picture {i}: Latitude={point[0]}, Longitude={point[1]}, Altitude={point[2]}")


CameraCalc


TypeError: string indices must be integers

In [6]:
!touch ~/catkin_ws_rm/src/quad_agent/path/waypoints.txt

In [7]:
!touch ~/catkin_ws_rm/src/quad_agent/path/img_waypoints.txt

In [8]:
with open('~/catkin_ws_rm/src/quad_agent/path/waypoints.txt', 'w') as f:
    for w in waypoints:
        f.write(f"{w}\n")

In [9]:
with open('~/catkin_ws_rm/src/quad_agent/path/img_waypoints.txt', 'w') as f:
    for w in image_capture_waypoints:
        f.write(f"{w}\n")